In [1]:
import sqlite3 as sql
import pandas as pd

In [110]:
df = pd.read_csv('./database.sqlite')

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


- CASEID - case number
- DOF1 - direction of force (highest)
- GAD1 - deformation location (highest)
- PSU - primary sampling unit number
- YEAR - calender year of crash
- STRATIF - case stratum
- BODYTYPE - vehicle body type
- DVEST - estimated highest delta V
- DVBASIS - basis for total deltav V (highest)
- DVTOTAL - total delta V
- MAKE - vehicle make
- MODELYR - vehicle model year
- AGE - age of occupant
- BAGAVAIL - air bag system availability
- BAGDEPLY - air bag system deployed
- BAGFAIL - air bag system failure
- CASENO - case sequence number
- CHTYPE - type of child safety seat
- DEATH - time of death
- HEIGHT - height of occupent
- HOPSTAY - hospital stay
- INJSEV - injury severity (police rating)
- MANAVAIL -
- MANFAIL - 
- MANUSE - manual belt system use 
- OCCNO - occupant number
- ROLE - occupant's role
- SEATPOS - occupant's seat position
- SEX - occupant's sex
- TREATMENT - treatment - mortality
- WEIGHT - occupant's weight

In [111]:
fatal = df.copy()

In [112]:
# changing all columns to lower case
columns = fatal.columns
columns_lower = []
for col in columns:
    columns_lower.append(col.lower())
fatal.columns = columns_lower

In [113]:
# dictionay of categorie ID's (named how they will appear when get dummies is applied)
dvest = {'dvest_0':'reconstructed_dv','dvest_1':'dv<10kmph','dvest_2':'10 >= dv < 25kmph','dvest_3':'25 >= dv < 40kmph',
         'divest_4':'40 >= dv <55kmph','divest_5':'dv >= 55kmph','divest_6':'minor damage','divest_7':'moderat damage',
         'divest_8':'severe damage'}
dvbasis = {'dvbasis_0':'not_ispctd','dvbasis_1':'S_dmg_only','dvbasis_2':'S_dmg_&_trjctry','dvbasis_3':'S_missing_vhcl', 
           'dvbasis_4':'S_dmg_cdc_only', 'dvbasis_5':'one_vhcle_byond_S_scope','dvbasis_6':'rollover',
           'dvbasis_7':'non_hrzntl_forces','dvbasis_8':'sideswipe_dmg','dvbasis_9':'severe_override',
          'dvbasis_10':'ylding_objct','dvbasis_11':'overlapping_dmg','dvbasis_12':'insuf_data',
           'dvbasis_98':'other','dvbasis_99':'unknown'}
gad1 = {'gad1_F':'front','gad1_L':'left','gad_R':'right','gad1_B':'back','gad1_uk':'unkown','gad1_T':'top',
        'gad1_U':'undercarriage'}
bagavail = {'bagavail_1':'no','bagavail_0':'yes','bagavail_2':'unknown'}
stratif = {'stratif_k':'fatality','stratif_F':'injry_no_tow_hsptl',
          'stratif_E':'injry_tow','stratif_C':'two_tow_no_hsptl','stratif_D'}

SyntaxError: invalid syntax (<ipython-input-113-1df5810dbc43>, line 14)

In [114]:
# replace 9 with uk for (unknown per the NASS documents)
fatal['gad1'] = fatal['gad1'].copy().replace('9','uk')

In [67]:
# replace 0 with 1 since they are the same thing according to the NASS documents
fatal['bagavail'] = fatal['bagavail'].copy().replace(0,1,axis=0)

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/pandas/core/generic.py:4480: UserWarning: the "axis" argument is deprecated and will be removed inv0.13; this argument has no effect
  warnings.warn('the "axis" argument is deprecated '


In [115]:
## removed unnamed: 0 column since it matched the index
# fatal.drop(labels='unnamed: 0',axis=1,inplace=True)

In [116]:
# removing letter at the end and going to drop caseno and PSU since 
# they "are identical variables across all NASS records"
fatal['caseid'] = fatal['caseid'].apply(lambda x: x[:-1].strip())

In [117]:
fatal.drop(['psu','caseno'],axis=1,inplace=True)

In [124]:
# Dictionary of each column's unique values key = counts, value = feature name
unique_dict = {}
for col in fatal.columns:
    unique_dict[col] =  fatal[col].nunique()

# Features <30 couts
categoricals = []
for key in unique_dict.keys():
    if unique_dict[key] < 30:
        categoricals.append(key)

In [126]:
for col in categoricals:
    print(col)
    print(fatal[col].value_counts())

gad1
F     1372618
L      411350
B      330703
R      319169
uk     100183
T       65536
U        3328
Name: gad1, dtype: int64
year
2004    544672
2006    500428
2005    494713
2007    243975
2008    195223
2009    181650
2010    157642
2011    150124
2012    104230
2014    103450
2013     22257
2015     18063
Name: year, dtype: int64
stratif
K    529817
F    442408
E    385249
C    236676
D    235326
G    230456
B    226083
H    214204
J    156951
A     59252
Name: stratif, dtype: int64
dvest
0.0    1674684
7.0     306520
6.0     131007
8.0      84874
2.0      12325
3.0       5534
1.0       3007
5.0       2872
4.0       1199
Name: dvest, dtype: int64
dvbasis
1.0     1035657
3.0      587972
11.0     301492
0.0      295200
4.0      109304
7.0       66250
5.0       53107
10.0      49190
98.0      40293
6.0       37035
9.0       32346
8.0       17075
2.0        1522
Name: dvbasis, dtype: int64
bagavail
1.0    1639519
0.0     662232
3.0       5951
2.0       5117
Name: bagavail, dtype: int

In [127]:
fatal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2716427 entries, 0 to 2716426
Data columns (total 29 columns):
caseid      object
dof1        float64
gad1        object
year        int64
stratif     object
bodytype    float64
dvest       float64
dvbasis     float64
dvtotal     float64
make        float64
modelyr     float64
age         float64
bagavail    float64
bagdeply    float64
bagfail     float64
chtype      float64
death       float64
height      float64
hospstay    float64
injsev      float64
manavail    float64
manfail     float64
manuse      float64
occno       float64
role        float64
seatpos     float64
sex         float64
treatmnt    float64
weight      float64
dtypes: float64(25), int64(1), object(3)
memory usage: 601.0+ MB


In [132]:
fatal[ fatal['bagavail']==0.0 ][['bagdeply','bagfail']]

,bagdeply,bagfail
12,0.0,0.0
17,0.0,0.0
22,0.0,0.0
27,0.0,0.0
32,0.0,0.0
37,0.0,0.0
42,0.0,0.0
47,0.0,0.0
52,0.0,0.0
57,0.0,0.0


In [ ]:
# Imputer

In [85]:
fatal.isnull().sum()

caseid            0
dof1         299352
gad1         113540
year              0
stratif           5
bodytype       2519
dvest        494405
dvbasis       89984
dvtotal     1091276
make          10659
modelyr       13077
age           56349
bagavail     403608
bagdeply     403608
bagfail      576251
chtype       284986
death        327097
height       689620
hospstay     394158
injsev        49824
manavail     411021
manfail      680669
manuse       495745
occno           165
role            200
seatpos      190062
sex           32035
treatmnt     331950
weight       654910
dtype: int64

In [93]:
fatal[ df['dvtotal'].isna() ]['year'].value_counts()

2006    197452
2004    190316
2005    184505
2007    104293
2008     84793
2009     80094
2010     69222
2011     65682
2012     53511
2014     45168
2013      9208
2015      7032
Name: year, dtype: int64